<a href="https://colab.research.google.com/github/2020-nlp-c/nlp-deeplearning/blob/master/quotation_kim/word2vec.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np
import pandas as pd

In [2]:
from nltk.tokenize import word_tokenize
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [3]:
doc = 'you will never know until you try'
token_list = word_tokenize(doc)
tokens = list(set(token_list))
tokens

['never', 'until', 'will', 'you', 'know', 'try']

In [4]:
from numpy import argmax
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder

# 문자열 라벨링
label_enc = LabelEncoder()
label_docs = label_enc.fit_transform(tokens)
# 바이너리 인코딩
onehot_enc = OneHotEncoder(sparse=False)
docs_label = label_docs.reshape(len(label_docs), 1) # n:1 matrix로 변환
onehot_docs = onehot_enc.fit_transform(docs_label)

In [5]:
onehot=np.identity(len(tokens)).tolist()

onehot_vector = []
for vec in onehot:
    onehot_vector.append([vec])
onehot_vector

[[[1.0, 0.0, 0.0, 0.0, 0.0, 0.0]],
 [[0.0, 1.0, 0.0, 0.0, 0.0, 0.0]],
 [[0.0, 0.0, 1.0, 0.0, 0.0, 0.0]],
 [[0.0, 0.0, 0.0, 1.0, 0.0, 0.0]],
 [[0.0, 0.0, 0.0, 0.0, 1.0, 0.0]],
 [[0.0, 0.0, 0.0, 0.0, 0.0, 1.0]]]

In [27]:
dic = dict(zip(tokens, onehot_vector))
dic

{'know': [[0.0, 0.0, 0.0, 0.0, 1.0, 0.0]],
 'never': [[1.0, 0.0, 0.0, 0.0, 0.0, 0.0]],
 'try': [[0.0, 0.0, 0.0, 0.0, 0.0, 1.0]],
 'until': [[0.0, 1.0, 0.0, 0.0, 0.0, 0.0]],
 'will': [[0.0, 0.0, 1.0, 0.0, 0.0, 0.0]],
 'you': [[0.0, 0.0, 0.0, 1.0, 0.0, 0.0]]}

In [53]:
onehot_lis = []
for token in token_list:
    for word in dic.keys():
        if token == word:
            onehot_lis.append(np.array(dic[word]))
onehot_lis

[array([[0., 0., 0., 1., 0., 0.]]),
 array([[0., 0., 1., 0., 0., 0.]]),
 array([[1., 0., 0., 0., 0., 0.]]),
 array([[0., 0., 0., 0., 1., 0.]]),
 array([[0., 1., 0., 0., 0., 0.]]),
 array([[0., 0., 0., 1., 0., 0.]]),
 array([[0., 0., 0., 0., 0., 1.]])]

In [42]:
df = pd.DataFrame(onehot_lis, columns=['onehot'], index=token_list)
df

,onehot
you,"[0.0, 0.0, 0.0, 1.0, 0.0, 0.0]"
will,"[0.0, 0.0, 1.0, 0.0, 0.0, 0.0]"
never,"[1.0, 0.0, 0.0, 0.0, 0.0, 0.0]"
know,"[0.0, 0.0, 0.0, 0.0, 1.0, 0.0]"
until,"[0.0, 1.0, 0.0, 0.0, 0.0, 0.0]"
you,"[0.0, 0.0, 0.0, 1.0, 0.0, 0.0]"
try,"[0.0, 0.0, 0.0, 0.0, 0.0, 1.0]"


In [54]:
win_size = 1
x = []
y = []

for i in range(len(onehot_lis)):
    tmp = []
    if i-win_size < 0 :
        tmp = onehot_lis[:i]+onehot_lis[(i+1):(i+win_size+1)]
    elif i+win_size >= len(onehot_lis) :
        tmp = onehot_lis[(i-win_size):i] + onehot_lis[(i+1):]
    else:
        tmp = onehot_lis[(i-win_size):i] + onehot_lis[(i+1):(i+win_size+1)]
    x.append(tmp)
x

[[array([[0., 0., 1., 0., 0., 0.]])],
 [array([[0., 0., 0., 1., 0., 0.]]), array([[1., 0., 0., 0., 0., 0.]])],
 [array([[0., 0., 1., 0., 0., 0.]]), array([[0., 0., 0., 0., 1., 0.]])],
 [array([[1., 0., 0., 0., 0., 0.]]), array([[0., 1., 0., 0., 0., 0.]])],
 [array([[0., 0., 0., 0., 1., 0.]]), array([[0., 0., 0., 1., 0., 0.]])],
 [array([[0., 1., 0., 0., 0., 0.]]), array([[0., 0., 0., 0., 0., 1.]])],
 [array([[0., 0., 0., 1., 0., 0.]])]]

In [55]:
x_new = []
for i in x:
    x_new.append(np.array(sum(i,[])))
x_new

ValueError: ignored

In [58]:
vector_dim = 4
w1 = np.random.rand(len(tokens),vector_dim)
w2 = np.random.rand(vector_dim, len(tokens))

h = []
for x_vecs in x:
    tmp = []
    for x_vector in x_vecs:
        tmp.append(np.dot(w1.T,x_vector.T))
    h.append(tmp)
h

[[array([[0.74160249],
         [0.4580865 ],
         [0.60421108],
         [0.25735334]])], [array([[0.44279025],
         [0.84531571],
         [0.50616838],
         [0.10297127]]), array([[0.04236217],
         [0.50878634],
         [0.60179188],
         [0.50075961]])], [array([[0.74160249],
         [0.4580865 ],
         [0.60421108],
         [0.25735334]]), array([[0.29460354],
         [0.82082632],
         [0.81617133],
         [0.46225117]])], [array([[0.04236217],
         [0.50878634],
         [0.60179188],
         [0.50075961]]), array([[0.93511161],
         [0.75170284],
         [0.10719605],
         [0.47677864]])], [array([[0.29460354],
         [0.82082632],
         [0.81617133],
         [0.46225117]]), array([[0.44279025],
         [0.84531571],
         [0.50616838],
         [0.10297127]])], [array([[0.93511161],
         [0.75170284],
         [0.10719605],
         [0.47677864]]), array([[0.49552667],
         [0.18686417],
         [0.12990718],
 

In [24]:
def softmax(x):
    e_x = np.exp(x - np.max(x))
    return e_x / e_x.sum()

In [59]:
y_hat = []
for h_vecs in h:
    tmp = []
    for h_vector in h_vecs:
        tmp.append(softmax(np.dot(w2.T,h_vector)))
    y_hat.append(tmp)
y_hat

[[array([[0.13055067],
         [0.20409991],
         [0.11728703],
         [0.22756147],
         [0.20906162],
         [0.11143931]])], [array([[0.12323521],
         [0.18327635],
         [0.11809747],
         [0.22177426],
         [0.20924402],
         [0.14437269]]), array([[0.16053079],
         [0.14246898],
         [0.11513579],
         [0.2556526 ],
         [0.16492724],
         [0.16128462]])], [array([[0.13055067],
         [0.20409991],
         [0.11728703],
         [0.22756147],
         [0.20906162],
         [0.11143931]]), array([[0.13416774],
         [0.15601492],
         [0.10076231],
         [0.27810261],
         [0.18389212],
         [0.14706029]])], [array([[0.16053079],
         [0.14246898],
         [0.11513579],
         [0.2556526 ],
         [0.16492724],
         [0.16128462]]), array([[0.1628695 ],
         [0.18789785],
         [0.1224259 ],
         [0.19387416],
         [0.24138498],
         [0.0915476 ]])], [array([[0.13416774],
   

In [60]:
loss = []
for predict in y_hat:
    tmp = []
    for j in range(len(predict)):
        tmp.append(np.log(predict[j]) * onehot_lis[i])
    loss.append(np.sum(tmp))
loss = -np.sum(loss)
loss

TypeError: ignored